# Session 12 - Measuring environmental impact

In this session, we're going to look at one particular way that we can measure the impact of our code on the world around us. In particular, we're going to be looking at how we can approximate the *environmental impact* of our cultural data science footprint.

To do this, we're going to use the open-source software package *CodeCarbon*. You can find more information at the following links:

- CodeCarbon Website: [https://codecarbon.io/](https://codecarbon.io/)
- GitHub Repo: [https://mlco2.github.io/codecarbon/](https://mlco2.github.io/codecarbon/)
- Documentation: [https://mlco2.github.io/codecarbon/](https://mlco2.github.io/codecarbon/)

We'll do some testing on HuggingFace pipelines.

## Testing HuggingFace pipelines

In [1]:
import os
from codecarbon import EmissionsTracker
from transformers import pipeline
import datasets
import pandas as pd
from tqdm.notebook import tqdm

2024-04-25 10:30:58.120367: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 10:30:58.124353: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 10:30:58.177361: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 10:30:59.150176: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


__Text summarization pipeline__

You may remember from a couple of weeks ago that *text summarization* was quite a compute intensive task. So let's see exactly how compute intensive it is.

In [2]:
text = """In this work, we presented the Transformer, the first sequence transduction model based entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention. 
For translation tasks, the Transformer can be trained significantly faster than architectures based on recurrent or convolutional layers. 
On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we achieve a new state of the art. 
In the former task our best model outperforms even all previously reported ensembles."""

In [3]:
summarizer = pipeline(task="summarization", 
                      min_length=10,
                      max_length=30)

No model was supplied, defaulted to google-t5/t5-small and revision d769bba (https://huggingface.co/google-t5/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

There are a number of different ways that we can work with CodeCarbon, all of which is clearly explained in the relevant documentation.

We'll go through each of them one at a time here.

## Method 1 - Creating a tracker object

In [4]:
tracker = EmissionsTracker()
tracker.start()
summary = summarizer(text)
tracker.stop()

[codecarbon INFO @ 10:31:51] [setup] RAM Tracking...
[codecarbon INFO @ 10:31:51] [setup] GPU Tracking...
[codecarbon INFO @ 10:31:51] No GPU found.
[codecarbon INFO @ 10:31:51] [setup] CPU Tracking...
[codecarbon WARNING @ 10:31:51] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:31:52] We saw that you have a Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:31:52] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
[codecarbon INFO @ 10:31:52] >>> Tracker's metadata:
[codecarbon INFO @ 10:31:52]   Platform system: Linux-5.4.256.el8-x86_64-with-glibc2.35
[codecarbon INFO @ 10:31:52]   Python version: 3.10.12
[codecarbon INFO @ 10:31:52]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 10:31:52]   Available RAM : 376.535 GB
[codecarbon INFO @ 10:31:52]   CPU count: 64
[codecarbon INFO @ 10:31:52]   CPU model: Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
[codecarbon INFO 

7.303957377374935e-05

## Method 2 - Context manager

In [5]:
with EmissionsTracker() as tracker:
    summary = summarizer(text)
    print(summary)

[codecarbon INFO @ 10:35:49] [setup] RAM Tracking...
[codecarbon INFO @ 10:35:49] [setup] GPU Tracking...
[codecarbon INFO @ 10:35:49] No GPU found.
[codecarbon INFO @ 10:35:49] [setup] CPU Tracking...
[codecarbon WARNING @ 10:35:49] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:35:50] We saw that you have a Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:35:50] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
[codecarbon INFO @ 10:35:50] >>> Tracker's metadata:
[codecarbon INFO @ 10:35:50]   Platform system: Linux-5.4.256.el8-x86_64-with-glibc2.35
[codecarbon INFO @ 10:35:50]   Python version: 3.10.12
[codecarbon INFO @ 10:35:50]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 10:35:50]   Available RAM : 376.535 GB
[codecarbon INFO @ 10:35:50]   CPU count: 64
[codecarbon INFO @ 10:35:50]   CPU model: Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
[codecarbon INFO 

[{'summary_text': 'the Transformer replaces recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention .'}]


## Method 3 - A Python decoractor



In [6]:
from codecarbon import track_emissions

@track_emissions
def summarization(text):
    summary = summarizer(text)
    print(summary)

In [7]:
summarization(text)

[codecarbon INFO @ 10:41:41] [setup] RAM Tracking...
[codecarbon INFO @ 10:41:41] [setup] GPU Tracking...
[codecarbon INFO @ 10:41:41] No GPU found.
[codecarbon INFO @ 10:41:41] [setup] CPU Tracking...
[codecarbon WARNING @ 10:41:41] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:41:42] We saw that you have a Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:41:42] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
[codecarbon INFO @ 10:41:42] >>> Tracker's metadata:
[codecarbon INFO @ 10:41:42]   Platform system: Linux-5.4.256.el8-x86_64-with-glibc2.35
[codecarbon INFO @ 10:41:42]   Python version: 3.10.12
[codecarbon INFO @ 10:41:42]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 10:41:42]   Available RAM : 376.535 GB
[codecarbon INFO @ 10:41:42]   CPU count: 64
[codecarbon INFO @ 10:41:42]   CPU model: Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
[codecarbon INFO 

[{'summary_text': 'the Transformer replaces recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention .'}]


## A more complex example

We can make the results more useful by changing the tracker parameters - full list can be found here [https://mlco2.github.io/codecarbon/parameters.html](https://mlco2.github.io/codecarbon/parameters.html).

In the example that follows, we're going to download a HuggingFace dataset and a pretrained emotion classification model. 

We also introduce specific *tasks* to more clearly understand the impact of different parts of our code.

In [9]:
outfolder = os.path.join("..", "emissions")
os.makedirs(outfolder, exist_ok=True)

tracker = EmissionsTracker(project_name="sentiment classification",
                           experiment_id="sentiment_classifier",
                           output_dir=outfolder,
                           output_file="emissions_sentiment.csv")

# tracking data downloading
tracker.start_task("load dataset")
dataset = datasets.load_dataset("imdb", 
                                split="test")
imdb_emissions = tracker.stop_task()

# tracking downloading and initializing model
tracker.start_task("build model")
classifier = pipeline(task="sentiment-analysis", 
                      model="cardiffnlp/twitter-roberta-base-emotion")
model_emissions = tracker.stop_task()

# tracking classification pipeline
tracker.start_task("run classification")
preds = []
for row in tqdm(dataset["text"][:1000]):
    preds.append(classifier(row[:100]))
classifier_emissions = tracker.stop_task()

tracker.stop()

[codecarbon INFO @ 10:48:14] [setup] RAM Tracking...
[codecarbon INFO @ 10:48:14] [setup] GPU Tracking...
[codecarbon INFO @ 10:48:14] No GPU found.
[codecarbon INFO @ 10:48:14] [setup] CPU Tracking...
[codecarbon WARNING @ 10:48:14] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:48:15] We saw that you have a Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:48:15] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
[codecarbon INFO @ 10:48:15] >>> Tracker's metadata:
[codecarbon INFO @ 10:48:15]   Platform system: Linux-5.4.256.el8-x86_64-with-glibc2.35
[codecarbon INFO @ 10:48:15]   Python version: 3.10.12
[codecarbon INFO @ 10:48:15]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 10:48:15]   Available RAM : 376.535 GB
[codecarbon INFO @ 10:48:15]   CPU count: 64
[codecarbon INFO @ 10:48:15]   CPU model: Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
[codecarbon INFO 

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

[codecarbon INFO @ 10:48:34] Energy consumed for RAM : 0.000625 kWh. RAM Power : 141.20075225830078 W
[codecarbon INFO @ 10:48:34] Energy consumed for all CPUs : 0.000188 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:48:34] 0.000813 kWh of electricity used since the beginning.


...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

[codecarbon INFO @ 10:48:43] Energy consumed for RAM : 0.000979 kWh. RAM Power : 141.20075225830078 W
[codecarbon INFO @ 10:48:43] Energy consumed for all CPUs : 0.000295 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:48:43] 0.001273 kWh of electricity used since the beginning.


  0%|          | 0/1000 [00:00<?, ?it/s]

[codecarbon WARNING @ 10:53:27] Background scheduler didn't run for a long period (284s), results might be inaccurate
[codecarbon INFO @ 10:53:27] Energy consumed for RAM : 0.012126 kWh. RAM Power : 141.20075225830078 W
[codecarbon INFO @ 10:53:27] Energy consumed for all CPUs : 0.003650 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:53:27] 0.015776 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:53:27] Energy consumed for RAM : 0.012126 kWh. RAM Power : 141.20075225830078 W
[codecarbon INFO @ 10:53:27] Energy consumed for all CPUs : 0.003650 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:53:27] 0.015776 kWh of electricity used since the beginning.
/home/ucloud/.local/lib/python3.10/site-packages/codecarbon/output.py:168: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exc

0.0028462549037700322

__Inspecting the results__

In [10]:
emissions_df = pd.read_csv("emissions.csv")

In [11]:
emissions_df["emissions"]

0    0.000073
1    0.000075
2    0.000072
Name: emissions, dtype: float64

## Tasks

- Now that you have the basics down, head over and consider Assignment 5!